In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import cnn               # CNN module
import pickle
from time import time

In [2]:
nn=cnn.conv_net()
w0,b0=nn.init_kernel_bias(num_inp_channels=3,kernel_size=3,num_kernels=64)

In [13]:
with open('saved_w8s.dump','rb') as f:
	w0,b0,w1,b1,w2,b2,w3,b3=pickle.load(f)

In [95]:
inp=inp.transpose(0,3,1,2)
inp.shape

(40, 32, 32, 3)

In [75]:
%timeit a=np.pad(inp,((0,0),(0,0),(1,1),(1,1)),'constant')

1.03 ms ± 38.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [113]:
a.shape

(40, 3, 34, 34)

In [125]:
%timeit conv0=conv2d(1,inp,w0,b0)

14.9 ms ± 729 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
conv0.shape

(40, 32, 32, 64)

In [121]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):      #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    for img in padded:     #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]), kernels.reshape(-1,kernels.shape[3])))
        out=(out+biases).reshape(out_row,out_col,kernels.shape[3])
        output.append(out)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [122]:
conv0=conv2d(1,inp,w0,b0)